This script can be used to generate the following events given some KPI constarints for one palnned date for one particular line:

*   FTT
*   Defective
*   Defects
*   Reject
*   Rectified
*   Manpower
*   Reject reasons
*   Change

The inputs we need are:

1. A plan for one day/ multipe days  for the line of interest
2. An hourly target table of KPIs for each day (date column and hour column both)
3. General schedule (Assuming we reuse the same schedule for all days)
4. An example template for each event type (just to order the columns)





   









In [1]:
# All imports
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import string
import uuid
from google.colab import drive

In [2]:
# Mount google drive and provide the file path that contains contraints in
# excel form
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

Mounted at /content/drive


In [3]:
# Read constraints sheet
xls = pd.read_excel(file_path,
                    sheet_name=["plan","schedule", "target_kpis",
                                "ftt", "defective", "reject", "defects",
                                "rectified", "manpower", "rr"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]
target_kpis = xls["target_kpis"]
ftt = xls["ftt"]
defective = xls["defective"]
reject = xls["reject"]
defects = xls["defects"]
rectified = xls["rectified"]
manpower = xls["manpower"]
reject_reason = xls["rr"]

In [4]:
# Timezone of the factory
hour_offset = 5.5

In [5]:
# Dictionary containing endline_qc id for each line
station_id = {"F1M7": "F1M7ST02", "F1M8": "F1M8ST02"}

In [6]:
# Function to generate random timestamps per-day that lies between a given start
# and end times

def generate_random_timestamps(start_time, end_time, min_increment,
                               max_increment, num_increments):
    """
    Generate a sorted list of timestamps with random increments for a
    specified date and time range.

    :param start_time: The start datetime.
    :param end_time: The end datetime.
    :param min_increment: Minimum increment in minutes.
    :param max_increment: Maximum increment in minutes.
    :param num_increments: Number of random increments to generate.
    :return: A DataFrame with the sorted timestamps.
    """
    # Generate random increments
    increments = [random.randint(min_increment, max_increment) for _ in
                  range(num_increments)]

    # Generate timestamps
    timestamps = []
    current_time = start_time

    while current_time < end_time:
        timestamps.append(current_time)
        increment = timedelta(seconds=random.choice(increments))
        current_time += increment
        if current_time > end_time:
            break

    # Ensure timestamps are sorted
    timestamps.sort()

    # Create a DataFrame
    df = pd.DataFrame({'ts': timestamps})

    return df

In [7]:
module_ids = plan["module_id"].unique()
module_ids

array(['F1M7', 'F1M8'], dtype=object)

In [8]:
all_dates = {}
for module_id in module_ids:
  all_dates[module_id] = []
  for date in plan[plan["module_id"] == module_id]["date"]:
    all_dates[module_id].append(date)
all_dates


{'F1M7': [Timestamp('2024-07-26 00:00:00'),
  Timestamp('2024-07-27 00:00:00'),
  Timestamp('2024-07-28 00:00:00'),
  Timestamp('2024-07-29 00:00:00'),
  Timestamp('2024-07-30 00:00:00'),
  Timestamp('2024-07-31 00:00:00'),
  Timestamp('2024-08-01 00:00:00')],
 'F1M8': [Timestamp('2024-07-26 00:00:00'),
  Timestamp('2024-07-27 00:00:00'),
  Timestamp('2024-07-28 00:00:00'),
  Timestamp('2024-07-29 00:00:00'),
  Timestamp('2024-07-30 00:00:00'),
  Timestamp('2024-07-31 00:00:00'),
  Timestamp('2024-08-01 00:00:00')]}

In [9]:
# Logic to determine function parameters(with an offset for factory loacl time)

dates ={}
starts = {}
ends={}
num_increments_dict = {}

for module_id in module_ids:

  date_list =[]
  start_time_list = []
  end_time_list = []
  num_increments_list = []

  for index in range(len(all_dates[module_id])):
    date = all_dates[module_id][index].to_pydatetime().date()
    start_time_str = pd.to_timedelta(schedule["start_time"].iloc[0]
                                     .strftime('%H:%M:%S'))- pd.Timedelta(
                                         hours=hour_offset)
    end_time_str = pd.to_timedelta(
        schedule["end_time"].iloc[-1].strftime('%H:%M:%S'))- pd.Timedelta(
            hours=hour_offset)

    start_time = datetime.combine(date, (datetime.min + start_time_str).time())
    end_time = datetime.combine(date, (datetime.min+ end_time_str).time())

    num_increments = plan["qty"].max()
    # print(date)
    # print(start_time)
    # print(end_time)
    # print(num_increments)

    date_list.append(date)
    start_time_list.append(start_time)
    end_time_list.append(end_time)
    num_increments_list.append(num_increments)

  dates[module_id] = date_list
  starts[module_id] = start_time_list
  ends[module_id] = end_time_list
  num_increments_dict[module_id] = num_increments_list



  #num_increments = 1000

min_increment = 2
max_increment = 6

In [10]:
# Verification
print(starts)


{'F1M7': [datetime.datetime(2024, 7, 26, 2, 0), datetime.datetime(2024, 7, 27, 2, 0), datetime.datetime(2024, 7, 28, 2, 0), datetime.datetime(2024, 7, 29, 2, 0), datetime.datetime(2024, 7, 30, 2, 0), datetime.datetime(2024, 7, 31, 2, 0), datetime.datetime(2024, 8, 1, 2, 0)], 'F1M8': [datetime.datetime(2024, 7, 26, 2, 0), datetime.datetime(2024, 7, 27, 2, 0), datetime.datetime(2024, 7, 28, 2, 0), datetime.datetime(2024, 7, 29, 2, 0), datetime.datetime(2024, 7, 30, 2, 0), datetime.datetime(2024, 7, 31, 2, 0), datetime.datetime(2024, 8, 1, 2, 0)]}


**FTT**

In [11]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)

ftts = {}

for module_id in module_ids:
  ts_df = pd.DataFrame()
  date_list = dates[module_id]
  starts_list = starts[module_id]
  ends_list= ends[module_id]
  num_increments_list = num_increments_dict[module_id]
  for i in range(len(date_list)):
    ts_df= pd.concat([ts_df, generate_random_timestamps(starts_list[i],
                                                        ends_list[i],
                                                        min_increment,
                                                        max_increment,
                                    num_increments_list[i])])
  ts_df['module_id'] = module_id
  ts_df.reset_index(drop=True, inplace=True)
  ftts[module_id] = ts_df

ftts['F1M7']

,ts,module_id
0,2024-07-26 02:00:00,F1M7
1,2024-07-26 02:00:06,F1M7
2,2024-07-26 02:00:09,F1M7
3,2024-07-26 02:00:13,F1M7
4,2024-07-26 02:00:18,F1M7
...,...,...
72806,2024-08-01 13:29:45,F1M7
72807,2024-08-01 13:29:47,F1M7
72808,2024-08-01 13:29:50,F1M7
72809,2024-08-01 13:29:52,F1M7


Now, I assume we have the trained synthesizers for each event type given below which we can load and use to generate more samples. But we may have to alter categorical data column values according to the plan's bctx and org. (So just learned numerical distributions are learned.)

In [12]:
!pip install sdv

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [13]:
# To import a synthesizer
import sdv
from sdv.single_table import CTGANSynthesizer

In [14]:
# Import the pretrained synthesizer for FTT
ftt_synthesizer = CTGANSynthesizer.load(
    filepath='/content/drive/My Drive/plan_data/ftt_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
# Generate some instances (to the same length as timestamps)
syn_ftts = {}
for module_id in ftts.keys():
  ts_df = ftts[module_id]
  synthetic_ftt = ftt_synthesizer.sample(num_rows=len(ts_df))
  synthetic_ftt['evt_gmt'] = ts_df['ts']
  synthetic_ftt['module_id'] = module_id
  syn_ftts[module_id] = synthetic_ftt


Sampling rows: 100%|██████████| 72766/72766 [00:06<00:00, 11478.43it/s]


In [16]:
# Function to determine the date based on a time threshold
def get_factory_date(dt):

    dt = dt + timedelta(hours=hour_offset)
    if dt.time() < pd.to_datetime('05:00:00').time():
        return (dt - timedelta(days=1)).date()
    else:
        return dt.date()

In [17]:
# Apply factory_date logic to evt_gmt
for module_id in syn_ftts.keys():
  synthetic_ftt = syn_ftts[module_id]
  synthetic_ftt['evt_gmt'] = pd.to_datetime(synthetic_ftt['evt_gmt'])
  synthetic_ftt['factory_date'] = synthetic_ftt['evt_gmt'].apply(get_factory_date)
  syn_ftts[module_id] = synthetic_ftt

In [18]:
# Verification
syn_ftts['F1M7'].head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,7,0,0,0,0,0,0
1,2024-07-26 02:00:06,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
2,2024-07-26 02:00:09,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
3,2024-07-26 02:00:13,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,8,0,0,0,0,0,0
4,2024-07-26 02:00:18,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0


In [19]:
# For the ease of manipulation, let us assume that we add piece by piece in
# all scenarios.
# This functions sets a value column to a given value.

def edit_value(df, column, value):
  df[column] = value

In [20]:
for module_id in syn_ftts.keys():
  synthetic_ftt = syn_ftts[module_id]
  edit_value(synthetic_ftt, "ftt_count", 1)
  syn_ftts[module_id] = synthetic_ftt

In [21]:
#syn_ftts['F1M7'].head()

In [22]:
# Find the seconds of the day for each event
def find_working_hour(column, hours):
  local_time = column + timedelta(hours=hours)
  return local_time.hour * 3600 + local_time.minute * 60 + local_time.second


In [23]:
for module_id in syn_ftts.keys():
  synthetic_ftt = syn_ftts[module_id]
  synthetic_ftt['factory_time'] = synthetic_ftt['evt_gmt'].apply(
      find_working_hour, args=(hour_offset,))
  syn_ftts[module_id] = synthetic_ftt


In [24]:
#syn_ftts['F1M8'].head()

In [25]:
# Add the seconds of the day to the schedule df
schedule['start_time'] = pd.to_datetime(schedule['start_time'],
                                        format='%H:%M:00').dt.time
schedule['seconds_of_day'] = schedule['start_time'].apply(lambda x: x.hour * 3600 + x.minute * 60)

In [26]:
# Verification
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [27]:
# Perform a backward-looking merge
for module_id in syn_ftts.keys():
  synthetic_ftt = syn_ftts[module_id]
  synthetic_ftt = synthetic_ftt.sort_values('factory_time')
  synthetic_ftt = pd.merge_asof(synthetic_ftt, schedule, left_on='factory_time',
                                right_on='seconds_of_day', direction='backward')


  synthetic_ftt.drop(columns=['factory_time', 'start_time', 'end_time',
                              'seconds_of_day'], inplace=True)
  synthetic_ftt = synthetic_ftt.rename(columns={'hour': 'Hour',
                                                'factory_date': 'Date',
                                                'module_id': 'Line'})
  syn_ftts[module_id] = synthetic_ftt



In [28]:
#syn_ftts['F1M8']

In [29]:
target_kpis

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Man power
0,2024-07-26,F1M7,1,132,8.28,3.20303,18,19,17,2,23,4,27
1,2024-07-26,F1M7,2,174,8.28,3.20303,21,23,7,18,23,5,28
2,2024-07-26,F1M7,3,175,8.28,3.20303,30,30,24,8,23,5,28
3,2024-07-26,F1M7,4,147,8.28,3.20303,25,25,23,6,23,5,28
4,2024-07-26,F1M7,5,223,8.28,3.20303,23,24,20,6,23,5,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,101,8.28,2.15303,18,19,9,12,14,6,20
150,2024-08-01,F1M8,8,119,8.28,2.15303,24,25,19,7,14,6,20
151,2024-08-01,F1M8,9,96,8.28,2.15303,17,19,17,2,14,6,20
152,2024-08-01,F1M8,10,125,8.28,2.15303,18,18,16,5,14,6,20


In [30]:
# Function to extract only the necessary attribute targets
def get_relevant_target(columns):
  targets = target_kpis[columns]
  return targets

In [31]:
# Traget for FTTs
targets_ftt = get_relevant_target(["Date","Line", "Hour", "FTT"])
targets_ftt[77:]

,Date,Line,Hour,FTT
77,2024-07-26,F1M8,1,82
78,2024-07-26,F1M8,2,118
79,2024-07-26,F1M8,3,88
80,2024-07-26,F1M8,4,83
81,2024-07-26,F1M8,5,116
...,...,...,...,...
149,2024-08-01,F1M8,7,101
150,2024-08-01,F1M8,8,119
151,2024-08-01,F1M8,9,96
152,2024-08-01,F1M8,10,125


In [32]:
def adjust_hourly_counts(df, target_counts, count_column, hour_column='Hour',
                         date_column='Date', module_column='Line'):
    """
    Adjust the counts in each hour, date, module_id group of a DataFrame to
    match the target counts.

    :param df: DataFrame containing hour, date, and count columns.
    :param target_counts: DataFrame with hour, date, and target_count columns.
    :param count_column: Name of the column containing count values.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param module_column: Name of the column containing module_id values.
    :return: DataFrame with adjusted counts.
    """

    # Ensure date and hour columns are of the same type (datetime64[ns])
    target_counts[date_column] = pd.to_datetime(target_counts[date_column])
    df[date_column] = pd.to_datetime(df[date_column])
    df[hour_column] = pd.to_datetime(df[hour_column], format='%H').dt.hour

    # Merge the dataframes on the 'hour','date','module_id' columns
    merged_df = df.merge(target_counts, on=[hour_column, date_column,
                                            module_column], how='left')

    # Create a list to store the adjusted groups
    result_groups = []

    # Iterate over each hour and date group
    for (date, hour, module), group in merged_df.groupby([date_column,
                                                          hour_column,
                                                          module_column]):
        # Calculate the difference between the current count and the target count
        current_count = len(group)
        target_count = group[count_column].iloc[0]
        difference = current_count - target_count
        #print(current_count, target_count)

        # Drop rows randomly to meet the target count
        if difference > 0:
            drop_indices = group.sample(n=difference).index
            group = group.drop(drop_indices)

        # Append the group to the result list
        result_groups.append(group)

    # Concatenate the adjusted groups into a single DataFrame
    result_df = pd.concat(result_groups, ignore_index=True)

    # Drop the 'target_count' column as it's no longer needed
    #result_df = result_df.drop(columns=[count_column])

    return result_df




In [33]:
for module_id in syn_ftts.keys():
  synthetic_ftt = syn_ftts[module_id]
  synthetic_ftt_regulated = adjust_hourly_counts(synthetic_ftt, targets_ftt,"FTT")
  synthetic_ftt_regulated = synthetic_ftt_regulated.reindex(columns=ftt.columns)
  synthetic_ftt_regulated['factory_date'] = synthetic_ftt_regulated['evt_gmt'].apply(get_factory_date)
  synthetic_ftt_regulated['module_id'] = module_id
  syn_ftts[module_id] = synthetic_ftt_regulated


In [34]:
plan_for_bctx = plan.groupby('module_id').first().reset_index()
plan_for_bctx

,module_id,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,F1M7,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,...,Print,2114,660,23,5,0,0,28,8.28,17503.92
1,F1M8,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,...,Print,1225,660,12,5,0,0,17,8.28,10143.00


In [35]:
def edit_org(df,columns,module_id):
  """
  Find the corresponding org values from plan per line

  :param df : Dataframe with synthetic data
  :param columns: List of columns to edit
  :param module_id: Line id
  :return: Edited dataframe
  """
  plan_row = plan_for_bctx[(plan_for_bctx["module_id"] == module_id)]
  for column in columns:
     df[column] = plan_row[column].iloc[0]
  return df


In [36]:
bctx_columns = [ "tenant_id", "tenant","cluster_id", "cluster", "factory_id",
                "factory","unit_id", "unit", "building_id","building","floor_id",
                 "floor", "zone_id", "zone", "section_id","section",
                 "department_id","department", "module_id","module", "stp"]

for module_id in syn_ftts.keys():
  synthetic_ftt_regulated = syn_ftts[module_id]
  synthetic_ftt_regulated =edit_org(synthetic_ftt_regulated,bctx_columns,module_id)
  syn_ftts[module_id] = synthetic_ftt_regulated



In [37]:
def edit_bctx(df,columns,module_id):

    """
    Find the corresponding bctx and org values from plan per line per day

    :param df : Dataframe with synthetic data
    :param columns: List of columns to edit
    :param module_id: Line id
    :return: Edited dataframe
    """

    # Ensure date_column is in datetime format
    df["factory_date"] = pd.to_datetime(df["factory_date"])
    plan["date"] = pd.to_datetime(plan["date"])

    current_plan = plan[plan["module_id"] == module_id]

    # Iterate through each row in the df
    for i in range(len(df)):
        # Get the date for the current row in df
        current_date = df.loc[i, "factory_date"]

        # Find the corresponding row in the plan DataFrame
        plan_row = current_plan[(current_plan["date"] == current_date) ]

        if not plan_row.empty:
            for column in columns:
                df.at[i, column] = plan_row.iloc[0][column]
        else:
            print(f"Date {current_date} not found in plan DataFrame")

    return df

In [38]:
# edit bctx and values as needed
bctx_columns = ["plan_id", "shift_id", "shift","buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

for module_id in syn_ftts.keys():
  synthetic_ftt_regulated = syn_ftts[module_id]
  synthetic_ftt_regulated =edit_bctx(synthetic_ftt_regulated,
                                            bctx_columns, module_id)
  syn_ftts[module_id] = synthetic_ftt_regulated

In [39]:
# Change the line_id manually
for module_id in syn_ftts.keys():
  synthetic_ftt_regulated = syn_ftts[module_id]
  synthetic_ftt_regulated['stp'] = synthetic_ftt_regulated['stp'] + '-' + station_id[module_id]
  synthetic_ftt_regulated['station_id'] = station_id[module_id]
  syn_ftts[module_id] = synthetic_ftt_regulated



In [40]:
synthetic_ftt_regulated = pd.concat(syn_ftts.values(), ignore_index=True)

In [41]:
#synthetic_ftt_regulated.to_csv('synthetic_ftt_regulated.csv', index=False)

In [42]:
synthetic_ftt_regulated

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:18,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
1,2024-07-26 02:01:02,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,L,ftt,1,0,0,0,0,0,0
2,2024-07-26 02:01:16,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
3,2024-07-26 02:01:37,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
4,2024-07-26 02:02:08,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18800,2024-08-01 13:27:32,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
18801,2024-08-01 13:28:09,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,XL,ftt,1,0,0,0,0,0,0
18802,2024-08-01 13:28:56,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,M,ftt,1,0,0,0,0,0,0
18803,2024-08-01 13:29:10,2024-08-01,13f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,L,ftt,1,0,0,0,0,0,0


**DEFECTIVES**

In [43]:
defectives = {}

for module_id in module_ids:
  ts_df = pd.DataFrame()
  date_list = dates[module_id]
  starts_list = starts[module_id]
  ends_list= ends[module_id]
  num_increments_list = num_increments_dict[module_id]
  for i in range(len(date_list)):
    ts_df= pd.concat([ts_df, generate_random_timestamps(starts_list[i],
                                                        ends_list[i],
                                                        min_increment,
                                                        max_increment,
                                    num_increments_list[i])])
  ts_df['module_id'] = module_id
  ts_df.reset_index(drop=True, inplace=True)
  defectives[module_id] = ts_df

In [44]:
# Import the pretrained synthesizer for defective
defective_synthesizer = CTGANSynthesizer.load(
    filepath='/content/drive/My Drive/plan_data/defective_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [45]:
# Generate some instances (to the same length as timestamps)
syn_defectives ={}
for module_id in defectives.keys():
  ts_defective = defectives[module_id]
  synthetic_defective = defective_synthesizer.sample(num_rows=len(ts_defective))
  synthetic_defective['evt_gmt'] = ts_defective['ts']
  synthetic_defective['evt_gmt'] = pd.to_datetime(synthetic_defective['evt_gmt'])
  synthetic_defective['factory_date'] = synthetic_defective['evt_gmt'].apply(get_factory_date)
  synthetic_defective['module_id'] = module_id
  syn_defectives[module_id] = synthetic_defective

Sampling rows: 100%|██████████| 72533/72533 [00:10<00:00, 6766.71it/s]


In [46]:
syn_defectives['F1M7'].head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-wnVsWx,0,1,1,0,0,0,0
1,2024-07-26 02:00:02,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-GrLoVq,0,1,1,0,0,0,0
2,2024-07-26 02:00:04,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-HRikVU,0,1,1,0,0,0,0
3,2024-07-26 02:00:06,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-DgKEuo,0,1,3,0,0,0,0
4,2024-07-26 02:00:12,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,L,defective,sdv-id-AQYBhy,0,1,1,0,0,0,0


In [47]:
# Edit defective value
for module_id in syn_defectives.keys():
  synthetic_defective = syn_defectives[module_id]
  edit_value(synthetic_defective, "defective_count", 1)
  syn_defectives[module_id] = synthetic_defective

In [48]:
# Function to generate gid
def generate_random_string(length=8):
    characters = string.ascii_uppercase + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

In [49]:
# Generate gid
for module_id in syn_defectives.keys():
  synthetic_defective = syn_defectives[module_id]
  synthetic_defective["gid"]= [generate_random_string() for _ in range(len(synthetic_defective))]
  synthetic_defective= synthetic_defective.drop_duplicates(subset=['gid'])
  synthetic_defective['factory_time'] = synthetic_defective['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
  syn_defectives[module_id] = synthetic_defective

In [50]:
# Perform a backward-looking merge
for module_id in syn_defectives.keys():
  synthetic_defective = syn_defectives[module_id]
  synthetic_defective = synthetic_defective.sort_values('factory_time')
  synthetic_defective = pd.merge_asof(synthetic_defective, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
  synthetic_defective.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)
  synthetic_defective = synthetic_defective.rename(columns={'hour': 'Hour',
                                                            'factory_date': 'Date',
                                                            'module_id': 'Line'})
  syn_defectives[module_id] = synthetic_defective




In [51]:
# Target for defectives
targets_defective = get_relevant_target(["Date","Line", "Hour",
                                         "Defective", "Defect"])
targets_defective

,Date,Line,Hour,Defective,Defect
0,2024-07-26,F1M7,1,18,19
1,2024-07-26,F1M7,2,21,23
2,2024-07-26,F1M7,3,30,30
3,2024-07-26,F1M7,4,25,25
4,2024-07-26,F1M7,5,23,24
...,...,...,...,...,...
149,2024-08-01,F1M8,7,18,19
150,2024-08-01,F1M8,8,24,25
151,2024-08-01,F1M8,9,17,19
152,2024-08-01,F1M8,10,18,18


In [52]:
# Adjust the defective count to tally with the given target
for module_id in syn_defectives.keys():
  synthetic_defective = syn_defectives[module_id]
  defective_adjusted = adjust_hourly_counts(synthetic_defective, targets_defective,
                                          "Defective")
  defective_adjusted.drop(columns=['Defective','Defect'], inplace=True)
  syn_defectives[module_id] = defective_adjusted




In [53]:
def distribute_defects(total_defects, num_defectives):
  """
    Distribute a given number of total defects between a specified number
    of defective events.

    :param total_sum: The total defects to be distributed.
    :param num_events: The number of defective events.
    :return: A list with the distributed values.
  """
  # Initial distribution: assign each event a base value

  base_defects = total_defects // num_defectives
  remaining = total_defects % num_defectives

  distribution = [base_defects] * num_defectives

  # Randomly distribute the remaining value
  for _ in range(remaining):
      distribution[np.random.randint(0, num_defectives)] += 1

  return distribution



In [54]:
def generate_hourly_counts(defectives, targets, hour_column='Hour',date_column= 'Date',line_column= 'Line', count_column='defect_count'):
    """
    Generate counts for each hour of each day group of a DataFrame to match the target counts by
    distributing the target sum among all rows in the hour + date group.

    :param defectives: DataFrame containing hour and count columns.
    :param targets: DataFrame with hour and target_count columns.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param line_column: Name of the column containing module_id values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with generated counts.
    """
    # Merge the dataframes on the 'hour and date' column
    merged_df = defectives.merge(targets, on=[date_column, hour_column,
                                              line_column], how='left')

    # Create a new column for generated counts
    merged_df['generated_count'] = 0

    # Iterate over each hour+date+line group
    for (date, hour, line), group in merged_df.groupby([date_column,
                                                        hour_column,
                                                        line_column]):
        # Get the target sum for this date + hour+line
        target_sum = group['Defect'].iloc[0]

        # Generate the new counts by distributing the target sum
        generated_counts = distribute_defects(target_sum, len(group))
        merged_df.loc[group.index, 'generated_count'] = generated_counts

    # # Drop the 'target_count' column as it's no longer needed
    # result_df = merged_df.drop(columns=['target_count'])

    return merged_df


In [55]:
for module_id in syn_defectives.keys():
  synthetic_defective = syn_defectives[module_id]
  merged_df = generate_hourly_counts(synthetic_defective, targets_defective)
  merged_df['defect_count']= merged_df['generated_count']
  merged_df= merged_df.rename(columns={'Date': 'factory_date',
                                       'Line': 'module_id'})
  syn_defectives[module_id] = merged_df



In [56]:
# Function to add unit_id, unit columns

def add_unit_columns(df):
    # Get the index of the 'department_id' column
    department_index = df.columns.get_loc('department_id')

    # Insert 'unit_id' column with null values before 'department_id'
    df.insert(department_index, 'unit', [None] * len(df))

    # Insert 'unit' column with null values before 'department_id' (again since 'department_id' index has shifted)
    df.insert(department_index, 'unit_id', [None] * len(df))

    return df

In [57]:
# edit bctx and values as needed
editable_org = ["tenant_id", "tenant","cluster_id", "cluster", "factory_id",
                "factory","unit_id", "unit", "building_id","building","floor_id",
                "floor", "zone_id", "zone","department_id","department",
                "section_id","section", "module_id","module", "stp" ]

editable_bctx = [ "plan_id", "shift_id", "shift",  "buyer_id","buyer","vendor_id",
                  "vendor", "po_id", "po", "so_id","so","li_id","li",
                  "fabric_type_id","fabric_type","product_category_id",
                  "product_category","product_type_id", "product_type",
                  "style_id", "style", "color_id", "color" ]

defects_for_defect={}

for module_id in syn_defectives.keys():
  synthetic_defective_regulated = syn_defectives[module_id]
  synthetic_defective_regulated = edit_org(synthetic_defective_regulated,
                                           editable_org,module_id)
  synthetic_defective_regulated = edit_bctx(synthetic_defective_regulated,
                                            editable_bctx,module_id)
  synthetic_defective_regulated ['stp'] = synthetic_defective_regulated ['stp'] + '-' +   station_id[module_id]
  synthetic_defective_regulated['station_id'] = station_id[module_id]
  defects_for_defect[module_id] = synthetic_defective_regulated.drop(
      columns=['generated_count','Defective', 'Defect'])
  merged_df = synthetic_defective_regulated.drop(
      columns=['generated_count','Hour', 'Defective', 'Defect'])
  syn_defectives[module_id] = merged_df

#merged_df.to_csv('synthetic_defective_regulated.csv', index=False)

In [58]:
synthetic_defective_regulated = pd.concat(syn_defectives.values(), ignore_index=True)


In [59]:
#synthetic_defective_regulated.to_csv('synthetic_defective_regulated.csv', index=False)

In [60]:
synthetic_defective_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'department_id', 'department', 'floor_id',
       'floor', 'zone_id', 'zone', 'section_id', 'section', 'module_id',
       'module', 'station_id', 'station', 'hsk', 'buf', 'apt', 'stp', 'stpv',
       'buyer_id', 'buyer', 'vendor_id', 'vendor', 'po_id', 'po', 'so_id',
       'so', 'li_id', 'li', 'fabric_type_id', 'fabric_type',
       'product_category_id', 'product_category', 'product_type_id',
       'product_type', 'style_id', 'style', 'color_id', 'color', 'size_id',
       'size', 'evn', 'gid', 'ftt_count', 'defective_count', 'defect_count',
       'reject_count', 'rectified_count', 'input_count', 'rw_defective_count',
       'unit_id', 'unit'],
      dtype='object')

In [61]:
synthetic_defective_regulated['defect_count'].sum()

3535

**DEFECTS**

In [62]:
# To create the defects dataframe, explode each of the  events in defective data,
# defect_count times
def generate_defects_base(columns):
  df_repeated = pd.DataFrame()
  for col in columns:
    repeated_df = []
    for index, row in synthetic_defective_regulated.iterrows():
        repeated_df.extend([row[col]] * row['defect_count'])
    df_repeated[col] = repeated_df
  return df_repeated

In [63]:
columns_for_defects = ["evt_gmt",	"factory_date",	"shift_id",	"shift",
                        "tenant_id",	"tenant",	"cluster_id",	"cluster",
                        "factory_id",	"factory",	"building_id",	"building",
                        "unit_id", "unit","department_id",	"department",
                        "floor_id",	"floor","zone_id",	"zone",	"section_id",
                         "section",	"module_id","module","stp","size",	"hsk",
                        "buf","apt","stpv","buyer_id","buyer","vendor_id",
                        "vendor","po_id","po","so_id","so","li_id","li",
                        "fabric_type_id",	"fabric_type",	"product_category_id",
                        "product_category",	"product_type_id",	"product_type",
                        "style_id",	"style","color_id",	"color","size_id","gid"]
defects_base = generate_defects_base(columns_for_defects)

In [64]:
# Findout already processed columns
added_columns = defects_base.columns

In [65]:
# # Findout what other columns should be in defects data

# for column_name in added_columns:
#     defects_temp = defects.drop(column_name, axis=1)
# defects_temp.head(200)

In [66]:
# Import the pretrained synthesizer for defective
defects_synthesizer = CTGANSynthesizer.load(
    filepath="/content/drive/My Drive/plan_data/defects_synthesizer.pkl"
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [67]:
# Generate synthetic dataset that is the same length as aggregated defective data
synthetic_defects = defects_synthesizer.sample(num_rows=len(defects_base))

Sampling rows: 100%|██████████| 3535/3535 [00:00<00:00, 11194.94it/s]


In [68]:
#Concatenate the aggregated defective data with generated defect-specific data
synthetic_defects = pd.concat([defects_base, synthetic_defects], axis=1)

In [69]:
# Manually change event type
synthetic_defects["evn"]= "defect"

In [70]:
# Reorder the data columns
synthetic_defects = synthetic_defects.reindex(columns=defects.columns)
#synthetic_defects.head(20)

In [71]:
#synthetic_defects.columns

In [72]:
#synthetic_defects.to_csv('sythetic_defects_regulated.csv', index=False)

**RECTIFIED**

In [73]:
# We are going to derive rectified as portion of events from the defectives
# with the working hour attached.
defectives = defects_for_defect.copy()
for module_id in defectives.keys():
  defectives[module_id].rename(columns={'factory_date': 'Date',
                                        'module_id': 'Line'}, inplace=True)


In [74]:
# Target for rectified
targets_rectified = get_relevant_target(["Date", "Hour", "Line", "Rectified"])
targets_rectified

,Date,Hour,Line,Rectified
0,2024-07-26,1,F1M7,17
1,2024-07-26,2,F1M7,7
2,2024-07-26,3,F1M7,24
3,2024-07-26,4,F1M7,23
4,2024-07-26,5,F1M7,20
...,...,...,...,...
149,2024-08-01,7,F1M8,9
150,2024-08-01,8,F1M8,19
151,2024-08-01,9,F1M8,17
152,2024-08-01,10,F1M8,16


In [75]:
def pick_rectified_events(all_defective, targets, hour_column='Hour',
                          date_column = 'Date', line_column='Line',
                          datetime_column='evt_gmt',
                          count_column='Rectified'):
    """
    Pick the first n events from the raw events DataFrame for each hour group based on the counts DataFrame.

    :param all_defective: DataFrame containing raw events with an hour and datetime column.
    :param targets: DataFrame containing the count of events per hour.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param line_column: Name of the column containing module_id values.
    :param datetime_column: Name of the column containing datetime values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with the selected events.
    """
    # Ensure the datetime column is of datetime type
    all_defective[datetime_column] = pd.to_datetime(all_defective[datetime_column])

    # Merge the raw_df and counts_df on the date + hour column
    merged_df = all_defective.merge(targets, on=[date_column, hour_column,
                                                 line_column], how='left')

    # Initialize an empty DataFrame to store the selected events
    selected_events = pd.DataFrame(columns=all_defective.columns)

    # # Iterate over each date+hour+line group
    # for (date, hour, line), group in merged_df.groupby([date_column, hour_column,
    #                                                     line_column]):
    #     # Get the target sum for this date + hour
    #     target_sum = group['defect_count'].iloc[0]

    # Iterate over each hour group
    for (date, hour, line), group in merged_df.groupby([date_column,
                                                        hour_column,
                                                        line_column]):
        # Get the target count for this hour
        target_count = group[count_column].iloc[0]

        # Sort the group by the datetime column
        sorted_group = group.sort_values(by=datetime_column)

        # Select the first n events
        selected_n_events = sorted_group.head(target_count)

        # Append the selected events to the result DataFrame
        selected_events = pd.concat([selected_events, selected_n_events])

    # Drop the count column from the result DataFrame
    #selected_events = selected_events.drop(columns=[count_column])

    return selected_events


In [76]:
# Convert both columns to datetime for merging
targets_rectified['Date'] = pd.to_datetime(targets_rectified['Date'])

for module_id in defectives.keys():
  defectives[module_id]['Date'] = pd.to_datetime(defectives[module_id]['Date'])


In [77]:
#defectives['F1M7'].columns

In [78]:
# Select certain defectives and transform them to rectified events
for module_id in defectives.keys():
  rectified_events = pick_rectified_events(defectives[module_id], targets_rectified)
  rectified_events.drop(columns=['Hour', 'Rectified'], inplace=True)
  rectified_events['evn'] = 'rectified'
  rectified_events['defective_count'] = 0
  rectified_events['defect_count'] = 0
  rectified_events['rectified_count'] = 1
  defectives[module_id] = rectified_events

<ipython-input-75-1d079b8df16e>:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_events = pd.concat([selected_events, selected_n_events])
<ipython-input-75-1d079b8df16e>:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_events = pd.concat([selected_events, selected_n_events])


In [79]:
# Function to increment a timestamp by 1-5 seconds randomly
def increment_timestamp(timestamp):
    random_seconds = random.randint(1, 5)
    return timestamp + timedelta(seconds=random_seconds)

In [80]:
# Apply the function to the 'timestamp' column and reorder the data columns
for module_id in defectives.keys():
  defectives[module_id]['evt_gmt'] = defectives[module_id]['evt_gmt'].apply(increment_timestamp)
  defectives[module_id].rename(columns={'Date': 'factory_date',
                                        'Line': 'module_id'}, inplace=True)
  #defectives[module_id]['stp'] = defectives[module_id]['stp'] + '-' + station_id[module_id]
  defectives[module_id] = defectives[module_id].reindex(columns=rectified.columns)


In [81]:
synthetic_rectified = pd.concat(defectives.values(), ignore_index=True)

In [82]:
#synthetic_rectified.to_csv('synthetic_rectified_regulated.csv', index=False)

**REJECT**

In [83]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)

rejects = {}

for module_id in module_ids:
  ts_df = pd.DataFrame()
  date_list = dates[module_id]
  starts_list = starts[module_id]
  ends_list= ends[module_id]
  num_increments_list = num_increments_dict[module_id]
  for i in range(len(date_list)):
    ts_df= pd.concat([ts_df, generate_random_timestamps(starts_list[i],
                                                        ends_list[i],
                                                        min_increment,
                                                        max_increment,
                                    num_increments_list[i])])
  ts_df['module_id'] = module_id
  ts_df.reset_index(drop=True, inplace=True)
  rejects[module_id] = ts_df



rejects['F1M7'].head()

,ts,module_id
0,2024-07-26 02:00:00,F1M7
1,2024-07-26 02:00:02,F1M7
2,2024-07-26 02:00:06,F1M7
3,2024-07-26 02:00:10,F1M7
4,2024-07-26 02:00:16,F1M7


In [84]:
# Import the pretrained synthesizer for Reject
reject_synthesizer = CTGANSynthesizer.load(
    filepath='/content/drive/My Drive/plan_data/reject_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [85]:
syn_rejects ={}
for module_id in rejects.keys():
  ts_rejects = rejects[module_id]
  synthetic_rejects = reject_synthesizer.sample(num_rows=len(ts_rejects))
  synthetic_rejects['evt_gmt'] = ts_rejects['ts']
  synthetic_rejects['evt_gmt'] = pd.to_datetime(synthetic_rejects['evt_gmt'])
  synthetic_rejects['factory_date'] = synthetic_rejects['evt_gmt'].apply(get_factory_date)
  synthetic_rejects['module_id'] = module_id
  synthetic_rejects["gid"]= [generate_random_string() for _ in range(len(synthetic_rejects))]
  synthetic_rejects['factory_time'] = synthetic_rejects['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
  syn_rejects[module_id] = synthetic_rejects

Sampling rows: 100%|██████████| 72513/72513 [00:08<00:00, 8388.12it/s]


In [86]:
# Target for rejects
targets_reject = get_relevant_target(["Date", "Hour", "Line", "Reject"])
targets_reject

,Date,Hour,Line,Reject
0,2024-07-26,1,F1M7,2
1,2024-07-26,2,F1M7,18
2,2024-07-26,3,F1M7,8
3,2024-07-26,4,F1M7,6
4,2024-07-26,5,F1M7,6
...,...,...,...,...
149,2024-08-01,7,F1M8,12
150,2024-08-01,8,F1M8,7
151,2024-08-01,9,F1M8,2
152,2024-08-01,10,F1M8,5


In [87]:
# Perform a backward-looking merge
for module_id in syn_rejects.keys():
  synthetic_reject = syn_rejects[module_id]
  synthetic_reject = synthetic_reject.sort_values('factory_time')
  synthetic_reject = pd.merge_asof(synthetic_reject, schedule,
                                   left_on='factory_time',
                                   right_on='seconds_of_day',
                                   direction='backward')
  synthetic_reject.drop(columns=['factory_time', 'start_time',
                                 'end_time', 'seconds_of_day'], inplace=True)
  synthetic_reject = synthetic_reject.rename(columns={'hour': 'Hour',
                                                      'factory_date': 'Date',
                                                      'module_id' :'Line'})
  syn_rejects[module_id] = synthetic_reject


In [88]:
# Adjust the reject count to tally with the given target
for module_id in syn_rejects.keys():
  synthetic_reject = syn_rejects[module_id]
  reject_adjusted = adjust_hourly_counts(synthetic_reject, targets_reject,
                                          "Reject")
  reject_adjusted.rename(columns={'Date': 'factory_date', 'Line': 'module_id'},
                         inplace=True)
  reject_adjusted =  reject_adjusted.reindex(columns=reject.columns)
  syn_rejects[module_id] = reject_adjusted


In [89]:
# edit bctx and values as needed
org_columns = [ "tenant_id", "tenant","cluster_id", "cluster", "factory_id",
               "factory","unit_id", "unit", "building_id","building",
                "department_id","department","floor_id", "floor", "zone_id",
                "zone", "section_id","section", "module_id","module", "stp",  ]

bctx_columns = ["plan_id", "shift_id", "shift","buyer_id","buyer","vendor_id",
                "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                "fabric_type","product_category_id", "product_category",
                "product_type_id", "product_type", "style_id", "style",
                "color_id", "color"]



for module_id in syn_rejects.keys():
  synthetic_reject_regulated = syn_rejects[module_id]
  synthetic_reject_regulated = edit_org(synthetic_reject_regulated,
                                         org_columns,module_id)
  synthetic_reject_regulated = edit_bctx(synthetic_reject_regulated,
                                          bctx_columns,module_id)
  synthetic_reject_regulated['stp'] = synthetic_reject_regulated['stp'] + '-' + station_id[module_id]
  synthetic_reject_regulated['station_id'] = station_id[module_id]
  edit_value(synthetic_reject_regulated, "reject_count", 1)
  syn_rejects[module_id] = synthetic_reject_regulated





In [90]:
synthetic_reject_regulated = pd.concat(syn_rejects.values(), ignore_index=True)

In [91]:
#synthetic_reject_regulated.to_csv('synthetic_reject_regulated.csv', index=False)

**MANPOWER**

Generate categorical data columns using for example, ftt events and generate the numbers directly using schedule.

In [92]:
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [93]:
def expand_schedule_with_dates(schedule_df, date_list):
    """
    Repeat the schedule DataFrame for each date in the date_list and add the relevant date.

    :param schedule_df: The schedule DataFrame with 11 hours.
    :param date_list: List of dates to repeat the schedule for.
    :return: A DataFrame with the schedule repeated for each date, with the date added to each row.
    """
    # Create an empty DataFrame to store the results
    expanded_df = pd.DataFrame()

    # Repeat the schedule DataFrame for each date
    for date in date_list:
        temp_df = schedule_df.copy()  # Copy the schedule DataFrame
        temp_df['Date'] = date  # Add the date column
        expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)

    return expanded_df

In [94]:
# Create a tweaked schedule per module
new_schedules = {}
for module_id in dates.keys():
  dates_of_module = dates[module_id]
  new_schedule = expand_schedule_with_dates(schedule, dates_of_module)
  new_schedule['module_id'] = module_id
  new_schedules[module_id] = new_schedule


In [95]:
# Combine the new schedules to one dataframe
new_schedule = pd.concat(new_schedules.values(), ignore_index=True)
#new_schedule

In [96]:
# Just using synthetic_ftt_regulated dataframe to extract the underlying org
manpower_cat = synthetic_ftt_regulated.head(len(new_schedule))
manpower_cat.drop(columns=['plan_id', 'evn', 'ftt_count', 'defective_count', 'defect_count',
                           'reject_count', 'rectified_count', 'input_count',
                           'rw_defective_count', 'buyer_id','buyer','vendor_id',
                           'vendor',	'po_id'	,'po',	'so_id',	'so',	'li_id'	,
                           'li',	'fabric_type_id',	'fabric_type',	'product_category_id',
                           'product_category',	'product_type_id',	'product_type',
                           'style_id',	'style',	'color_id',	'color'	,'size_id',	'size'], inplace=True)


In [97]:
# Iterate over each datetime object in the Series and format it

start_time_str = [(dt.strftime('%H:%M:%S')) for dt in new_schedule["start_time"]]
ts_manpower = pd.DataFrame()
ts_manpower['start_time'] = start_time_str
ts_manpower['module_id'] = new_schedule['module_id']
#ts_manpower





In [98]:
# Generate timestamps for manpower eventsand tweak major columns for a future merge
manpower_cat['evt_gmt'] = pd.to_datetime(new_schedule['Date'].astype(str) + ' ' +ts_manpower['start_time'] )
manpower_cat['module_id'] = new_schedule['module_id']
manpower_cat['factory_date']= manpower_cat['evt_gmt'].apply(get_factory_date)
manpower_cat['Hour'] = new_schedule['hour']
#manpower_cat

In [99]:
# Target for defectives (This example target does not have ironman and qc)
targets_manpower = get_relevant_target(["Date", "Hour","Line", "Operator", "Helper", "Man power"
                                        #"Iron man", "QC"
                                        ])
targets_manpower

,Date,Hour,Line,Operator,Helper,Man power
0,2024-07-26,1,F1M7,23,4,27
1,2024-07-26,2,F1M7,23,5,28
2,2024-07-26,3,F1M7,23,5,28
3,2024-07-26,4,F1M7,23,5,28
4,2024-07-26,5,F1M7,23,5,28
...,...,...,...,...,...,...
149,2024-08-01,7,F1M8,14,6,20
150,2024-08-01,8,F1M8,14,6,20
151,2024-08-01,9,F1M8,14,6,20
152,2024-08-01,10,F1M8,14,6,20


In [100]:
# lookup the hourly values from given target
manpower_cat.rename(columns={'factory_date': 'Date', 'module_id': 'Line'}, inplace=True)
manpower_cat['Date'] = pd.to_datetime(manpower_cat['Date'])

# Ensure 'Date' in targets_manpower is also datetime
targets_manpower['Date'] = pd.to_datetime(targets_manpower['Date'])


synthetic_manpower = manpower_cat.merge(targets_manpower, on=["Date", 'Hour', 'Line'], how='left')
synthetic_manpower.drop(columns=['Hour'], inplace=True)
synthetic_manpower.rename(columns={'Operator': 'operator', 'Helper': 'helper', 'Man power': 'manpower'}, inplace=True)
synthetic_manpower

,evt_gmt,Date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,station_id,station,hsk,buf,apt,stp,stpv,operator,helper,manpower
0,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,23,4,27
1,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,23,5,28
2,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,23,5,28
3,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,23,5,28
4,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,23,5,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01 14:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,14,6,20
150,2024-08-01 15:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,14,6,20
151,2024-08-01 16:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,14,6,20
152,2024-08-01 17:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,PT.Purnama Asih Sur,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,14,6,20


In [101]:
# Add missing column and values if not present in target
if "iron man" not in synthetic_manpower.columns:
    synthetic_manpower["ironman"] = 0
if "qc" not in synthetic_manpower.columns:
    synthetic_manpower["qc"] = 0

In [102]:
# Reformatting columns
synthetic_manpower.rename(columns={'Date': 'factory_date',
                                   'Line': 'module_id'}, inplace=True)

In [103]:
# edit bctx and values as needed
editable_bctx = ['shift_id', 'shift']
editable_org = ['tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'unit_id', 'unit', 'department_id', 'department', 'floor_id', 'floor',
       'zone_id', 'zone', 'section_id', 'section', 'module_id', 'module',"stp"
       ]

syn_manpower = {}
# Breaking up the synthetic_manpower dataframe into modulewise cases and
# editing bctx and org
# Also adjust the time to gmt
for module_id in synthetic_manpower.module_id.unique():
  modulewise_mp = synthetic_manpower[synthetic_manpower['module_id'] == module_id]
  modulewise_mp.reset_index(drop=True, inplace=True)
  modulewise_mp = edit_bctx(modulewise_mp, editable_bctx, module_id)
  modulewise_mp = edit_org(modulewise_mp, editable_org, module_id)
  modulewise_mp['stp'] = modulewise_mp['stp'] +  '-' + station_id[module_id]
  modulewise_mp['station_id'] = station_id[module_id]
  modulewise_mp['evt_gmt'] = pd.to_datetime(modulewise_mp['evt_gmt'])
  modulewise_mp['evt_gmt'] = modulewise_mp['evt_gmt'] - pd.to_timedelta(
      hour_offset, unit='h')
  modulewise_mp = modulewise_mp.reindex(columns=manpower.columns)
  syn_manpower[module_id] = modulewise_mp

In [104]:
synthetic_manpower= pd.concat(syn_manpower.values(), ignore_index=True)

In [105]:
#synthetic_manpower.to_csv('synthetic_manpower_regulated.csv', index=False)

**Now we are going to generate an eid for all synthetic data**

In [106]:
def add_unique_event_ids_and_save(*dfs, filenames):
    """
    Add unique event_id column to multiple DataFrames and save each to a CSV file.

    Parameters:
    *dfs : list of pd.DataFrame
        List of DataFrames to which unique event_ids will be added.
    filenames : list of str
        List of filenames corresponding to each DataFrame for saving as CSV.

    Returns:
    list of pd.DataFrame
        List of DataFrames with unique event_ids added.
    """

    # Calculate the total number of rows across all DataFrames
    total_rows = sum(len(df) for df in dfs)

    # Generate unique event_ids
    event_ids = [str(uuid.uuid4()) for _ in range(total_rows)]

    # Initialize the start index for slicing event_ids
    start_idx = 0

    # Function to move the last column to the first position
    def move_last_column_to_first(df):
        cols = df.columns.tolist()
        new_cols = [cols[-1]] + cols[:-1]
        return df[new_cols]

    # Assign unique event_ids to each DataFrame and save to CSV
    for df, filename in zip(dfs, filenames):
        end_idx = start_idx + len(df)
        df['eid'] = event_ids[start_idx:end_idx]
        start_idx = end_idx
        df = move_last_column_to_first(df)
        df.to_csv(filename, index=False)





# Filenames for saving the CSVs
filenames = ['regulated_synthetic_ftt.csv', 'regulated_synthetic_defective.csv',
             'regulated_synthetic_defect.csv', 'regulated_synthetic_rectified.csv',
             'regulated_synthetic_reject.csv','regulated_synthetic_manpower.csv']



# Add unique event_ids to DataFrames and save to CSVs
add_unique_event_ids_and_save(synthetic_ftt_regulated,synthetic_defective_regulated,
                              synthetic_defects,synthetic_rectified,
                              synthetic_reject_regulated,synthetic_manpower
                              ,filenames=filenames)



In [107]:
# Adding plan to a seperate csv file
plan.to_csv('plan.csv', index=False)

In [108]:
# Function to bring the added eid to the very begining
def move_last_column_to_front(df):
    """
    Moves the last column of the DataFrame to the front.

    Parameters:
    df (pd.DataFrame): The DataFrame to be processed.

    Returns:
    pd.DataFrame: The DataFrame with the last column moved to the front.
    """
    # Get the last column name
    last_column = df.columns[-1]

    # Reorder columns with the last column first
    new_order = [last_column] + list(df.columns[:-1])

    # Reindex the DataFrame
    df = df[new_order]

    return df


In [109]:
# Drop gid if present and combine ftt,reject,defective and rectified as production data


def generate_production_dataset(df_list, column_name):
    """
    Checks if a particular column exists in each DataFrame in the list,
    drops that column if it exists, and then merges all DataFrames vertically.

    Parameters:
    df_list (list of pd.DataFrame): List of DataFrames to be processed.
    column_name (str): The name of the column to check and drop.

    Returns:
    pd.DataFrame: The merged DataFrame.
    """
    processed_dfs = []

    for df in df_list:
        if column_name in df.columns:
            df = df.drop(columns=[column_name])
        processed_dfs.append(df)

    # Concatenate all DataFrames vertically
    merged_df = pd.concat(processed_dfs, ignore_index=True)
    merged_df = move_last_column_to_front(merged_df)

    merged_df.to_csv('production.csv', index=False)



In [110]:
# Generate production dataset

df_list = [synthetic_ftt_regulated,synthetic_defective_regulated,
           synthetic_rectified,synthetic_reject_regulated]
column_name = 'gid'
generate_production_dataset(df_list, column_name)

**REJECT REASON**

In [111]:
synthetic_reject_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'gid', 'evn', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count', 'eid'],
      dtype='object')

In [112]:
rejects_for_rr = synthetic_reject_regulated.copy()
rejects_for_rr.drop(columns=['evn', 'ftt_count','defective_count',
                             'defect_count', 'reject_count', 'rectified_count',
                             'input_count', 'rw_defective_count'], inplace=True)
rejects_for_rr.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,product_type_id,product_type,style_id,style,color_id,color,size_id,size,gid,eid
0,2024-07-26 02:33:53,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,M,YJ07TGFW,8e28c15b-deed-4160-93ea-13ea6b1c0c2e
1,2024-07-26 02:37:31,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,XL,TRF0JR5L,0ffb78b1-78df-43c2-a38a-ee6d913a59a9
2,2024-07-26 03:02:01,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,L,L4PW5CCB,b3bfc5d8-3184-4cf3-8e6f-60b630bcceb6
3,2024-07-26 03:04:16,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,XL,UVMSZ2C6,9a2dae5d-5695-48e4-b9e4-1eb537f2f744
4,2024-07-26 03:04:42,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Top,1,3667M,1,Print,1,L,111X00I7,1c64e28e-d22c-4116-8786-ccbd0ffde526


In [113]:
# Import the pretrained synthesizer for reject reasons
rr_synthesizer = CTGANSynthesizer.load(
    filepath='/content/drive/My Drive/plan_data/rr_synthesizer.pkl'
)

In [114]:
# Generate a set of synthetic reject reasons
synthetic_rr = rr_synthesizer.sample(num_rows=len(rejects_for_rr))


Sampling rows: 100%|██████████| 1309/1309 [00:00<00:00, 15808.42it/s]


In [115]:
synthetic_rr_regulated = pd.concat([rejects_for_rr, synthetic_rr], axis=1)
synthetic_rr_regulated.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,color_id,color,size_id,size,gid,eid,sign,reject_reason_id,reject_reason_code,reject_reason
0,2024-07-26 02:33:53,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,M,YJ07TGFW,8e28c15b-deed-4160-93ea-13ea6b1c0c2e,1,5,304,Fabric Defect
1,2024-07-26 02:37:31,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,XL,TRF0JR5L,0ffb78b1-78df-43c2-a38a-ee6d913a59a9,1,5,304,Fabric Defect
2,2024-07-26 03:02:01,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,L,L4PW5CCB,b3bfc5d8-3184-4cf3-8e6f-60b630bcceb6,1,2,301,Uneven
3,2024-07-26 03:04:16,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,XL,UVMSZ2C6,9a2dae5d-5695-48e4-b9e4-1eb537f2f744,1,4,303,Needle Defect
4,2024-07-26 03:04:42,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,1,Print,1,L,111X00I7,1c64e28e-d22c-4116-8786-ccbd0ffde526,1,9,308,Other


In [116]:
eid_column = synthetic_rr_regulated['eid']

In [117]:
synthetic_rr_regulated = synthetic_rr_regulated.reindex(columns=reject_reason.columns)
synthetic_rr_regulated = pd.concat([eid_column, synthetic_rr_regulated], axis=1)
synthetic_rr_regulated.head()

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,style,color_id,color,size_id,size,sign,gid,reject_reason_id,reject_reason_code,reject_reason
0,8e28c15b-deed-4160-93ea-13ea6b1c0c2e,2024-07-26 02:33:53,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,M,1,YJ07TGFW,5,304,Fabric Defect
1,0ffb78b1-78df-43c2-a38a-ee6d913a59a9,2024-07-26 02:37:31,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,XL,1,TRF0JR5L,5,304,Fabric Defect
2,b3bfc5d8-3184-4cf3-8e6f-60b630bcceb6,2024-07-26 03:02:01,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,L,1,L4PW5CCB,2,301,Uneven
3,9a2dae5d-5695-48e4-b9e4-1eb537f2f744,2024-07-26 03:04:16,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,XL,1,UVMSZ2C6,4,303,Needle Defect
4,1c64e28e-d22c-4116-8786-ccbd0ffde526,2024-07-26 03:04:42,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,3667M,1,Print,1,L,1,111X00I7,9,308,Other


In [118]:
synthetic_rr_regulated.to_csv('regulated_synthetic_rr.csv', index=False)

**CHANGE**

In [119]:
# Load synthetic manpower dataset
synthetic_manpower = pd.read_csv('regulated_synthetic_manpower.csv')

In [120]:
synthetic_manpower['change_type'] = 'manpower'
synthetic_manpower['scope'] = 'production'
synthetic_manpower['change'] = synthetic_manpower.apply(lambda row: {
        "ironman": str(row['ironman']),
        "helper": str(row['helper']),
        "operator": str(row['operator'])
}, axis=1)

In [121]:
synthetic_manpower.drop(columns=['manpower', 'operator', 'helper', 'ironman',
       'qc'], inplace=True)

In [122]:
synthetic_manpower.head()

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,station_id,station,hsk,buf,apt,stp,stpv,change_type,scope,change
0,429f7661-6c4d-46c3-ac56-ca6770067c75,2024-07-26 02:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '4', 'operator': '23'}"
1,47ee6619-b12f-423a-abb1-f980c292bba3,2024-07-26 03:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '23'}"
2,aa9ad674-6390-4b55-bd39-995d3169386a,2024-07-26 04:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '23'}"
3,84d8e2a4-9f15-4ba2-ad2b-06da75940864,2024-07-26 05:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '23'}"
4,d14334da-6d06-47ae-9ebc-7309c6bc4e6d,2024-07-26 06:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,manpower,production,"{'ironman': '0', 'helper': '5', 'operator': '23'}"


In [123]:
# Copy the same skeleton as manpower change
synthetic_bctx_temp = synthetic_manpower.copy()

In [124]:
# Convert factort_date to a datetime for a plan merge
synthetic_bctx_temp['factory_date'] = pd.to_datetime(synthetic_bctx_temp['factory_date'])

In [125]:
# Merge plan and change dataframes on plan date and get the bctx in the required
# nested format
synthetic_bctx= pd.merge(synthetic_bctx_temp, plan[['date','module_id','buyer','po','style',
                                                    'color']], how='left',
                         left_on=['factory_date', 'module_id'], right_on=['date','module_id'])
synthetic_bctx['change_type'] = 'running_order'
synthetic_bctx['scope'] = 'application'
synthetic_bctx['change'] = synthetic_bctx.apply(lambda row: {
    "buyer": row['buyer'],
    "po": row['po'],
    "style": row['style'],
    "color": row['color']
}, axis=1)
synthetic_bctx.drop(columns=['date','buyer','po','style','color'], inplace=True)
synthetic_bctx['factory_date'] = synthetic_bctx['factory_date'].dt.date
synthetic_bctx.head(25)

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,station_id,station,hsk,buf,apt,stp,stpv,change_type,scope,change
0,429f7661-6c4d-46c3-ac56-ca6770067c75,2024-07-26 02:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
1,47ee6619-b12f-423a-abb1-f980c292bba3,2024-07-26 03:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
2,aa9ad674-6390-4b55-bd39-995d3169386a,2024-07-26 04:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
3,84d8e2a4-9f15-4ba2-ad2b-06da75940864,2024-07-26 05:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
4,d14334da-6d06-47ae-9ebc-7309c6bc4e6d,2024-07-26 06:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
5,270df97c-6d67-4cee-a56a-07c9e47dca0b,2024-07-26 07:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
6,f803f3b5-2bd9-42a4-b289-6bee21c48b38,2024-07-26 08:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
7,2a8e6695-61c0-43dc-a443-26089dc59bd9,2024-07-26 09:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
8,09362c00-b7c2-495c-bf02-ffc02653d50e,2024-07-26 10:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."
9,9cebb8c2-d1b6-4655-9733-b2089a7e0343,2024-07-26 11:30:00,2024-07-26,1,Morning,NaN,NaN,NaN,NaN,ID_PAS,...,F1M7ST02,Station 2,primary,out,endline_qc,ID_PAS-SW-BD1-F1-F1M7-F1M7ST02,1.0.0,running_order,application,"{'buyer': 'M&S', 'po': 2011242746, 'style': '3..."


In [126]:
regulated_synthetic_change = pd.concat([synthetic_manpower, synthetic_bctx], axis=0)
regulated_synthetic_change.to_csv('regulated_synthetic_change.csv', index=False)

In [127]:
len(regulated_synthetic_change)

308